# Лабораторна робота № 4

## ІП-14 Бабіч Денис (09.07.2003)

---

# Підготовчий етап

## Імпортування необхідних модулів

In [1]:
import cv2
import numpy as np

## Створення необхідних функцій

In [2]:
def preprocess_frame(frame: np.ndarray, blur_strength: int, threshold: float, *roi_polygons: list) -> np.ndarray: 
    preprocessed_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    preprocessed_frame = cv2.equalizeHist(preprocessed_frame)

    preprocessed_frame = cv2.GaussianBlur(preprocessed_frame, (blur_strength, blur_strength), 0)

    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (1, 1))
    preprocessed_frame = cv2.morphologyEx(preprocessed_frame, cv2.MORPH_CLOSE, kernel)
    preprocessed_frame = cv2.erode(preprocessed_frame, kernel, iterations = 1)
    preprocessed_frame = cv2.dilate(preprocessed_frame, kernel, iterations = 1)

    preprocessed_frame = cv2.Canny(preprocessed_frame, threshold / 2, threshold)

    mask = np.zeros_like(preprocessed_frame)

    for roi_polygon_vertices in roi_polygons:
        cv2.fillPoly(mask, [np.array(roi_polygon_vertices, dtype = np.int32)], 255)

    preprocessed_frame = cv2.bitwise_and(preprocessed_frame, mask)

    return preprocessed_frame

def determine_obejcts_rectangles(frame: np.ndarray, width_min: int, width_max: int, height_min: int, height_max: int, aspect_ratio_min: float, aspect_ratio_max: float) -> list:
    objects_rectangles = []

    contours, _ = cv2.findContours(frame, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for contour in contours:
        x, y, width, height = cv2.boundingRect(contour)
        aspect_ratio = float(width) / height

        if (width_min < width < width_max) and (height_min < height < height_max) and (aspect_ratio_min < aspect_ratio < aspect_ratio_max):
            objects_rectangles.append((x, y, width, height))

    objects_rectangles = [r1 for r1 in objects_rectangles if not any((r1[0] > r2[0] and r1[1] > r2[1] and r1[0] + r1[2] < r2[0] + r2[2] and r1[1] + r1[3] < r2[1] + r2[3] and r1[2] * r1[3] < r2[2] * r2[3]) for r2 in objects_rectangles if r1 != r2)]

    return objects_rectangles

def overlay_objects_rectangles(frame: np.ndarray, rectangles: list, roi_title: str = "", color: tuple = (0, 255, 0), thickness: int = 2) -> np.ndarray:
    ROI_TITLE_OFFSET = 10

    FONT_SCALE = 0.5
    FONT_LINE_TYPE = 2

    for (x, y, width, height) in rectangles:
        cv2.rectangle(frame, (x, y), (x + width, y + height), color, thickness)

        if roi_title:
            cv2.putText(frame, roi_title, (x, y - ROI_TITLE_OFFSET), cv2.FONT_HERSHEY_SIMPLEX, FONT_SCALE, color, FONT_LINE_TYPE)

    return frame

---

# Основний етап

## Застосування на прикладі відео

In [3]:
KEY_ESCAPE = 27

CAPTURE = cv2.VideoCapture("video.mp4")
FPS = int(CAPTURE.get(cv2.CAP_PROP_FPS))

THRESHOLD = 250
BLUR_STRENGTH = 5

WIDTH_MIN = 55
WIDTH_MAX = 110
HEIGHT_MIN = 30
HEIGHT_MAX = 55
ASPECT_RATIO_MIN = 1.25
ASPECT_RATIO_MAX = 3.75

FRAME_HEIGHT, FRAME_WIDTH = CAPTURE.read()[1].shape[:2]

FRAME_HALF_HEIGHT = FRAME_HEIGHT / 2

OFFSET_MARKUP = 45
OFFSET_TOP_BOUNDS = 110
OFFSET_BOTTOM_BOUNDS = 75

ROI_TOP_ROAD = [(0, OFFSET_TOP_BOUNDS),
                (0, FRAME_HALF_HEIGHT - OFFSET_MARKUP),
                (FRAME_WIDTH, FRAME_HALF_HEIGHT - OFFSET_MARKUP),
                (FRAME_WIDTH, OFFSET_TOP_BOUNDS)]

ROI_BOTTOM_ROAD = [(0, FRAME_HEIGHT - OFFSET_BOTTOM_BOUNDS),
                   (0, FRAME_HALF_HEIGHT + OFFSET_MARKUP),
                   (FRAME_WIDTH, FRAME_HALF_HEIGHT + OFFSET_MARKUP),
                   (FRAME_WIDTH, FRAME_HEIGHT - OFFSET_BOTTOM_BOUNDS)]

ROI_TITLE = "Car"

while CAPTURE.isOpened():
    ret, frame = CAPTURE.read()

    if not ret:
        break

    preprocessed_frame = preprocess_frame(frame, BLUR_STRENGTH, THRESHOLD, ROI_TOP_ROAD, ROI_BOTTOM_ROAD)
    objects_roi = determine_obejcts_rectangles(preprocessed_frame, WIDTH_MIN, WIDTH_MAX, HEIGHT_MIN, HEIGHT_MAX, ASPECT_RATIO_MIN, ASPECT_RATIO_MAX)

    processed_frame = overlay_objects_rectangles(frame, objects_roi, ROI_TITLE)

    cv2.imshow("", processed_frame)

    if cv2.waitKey(FPS) & 0xFF == KEY_ESCAPE:
        break

CAPTURE.release()
cv2.destroyAllWindows()